In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib import cm
import netCDF4
from netCDF4 import num2date
from dateutil import parser, tz
from datetime import datetime
from ncdump import ncdump
from subprocess import call
import os


def getmodel(select_model, mydate, cycle):
    """Retrieve URL from the NOMADS/OpenDAP server
    and return file
    """
    model = {
        'gfs'  : 'gfs_0p25/gfs'+mydate+'/gfs_0p25_'+cycle+'z',
        'arw'  : 'hiresw/hiresw'+mydate+'/hiresw_conusarw_'+cycle+'z',
        'nmm'  : 'hiresw/hiresw'+mydate+'/hiresw_conusnmmb_'+cycle+'z',
        'hrrr' : 'hrrr/hrrr'+mydate+'/hrrr_sfc_'+cycle+'z',
        'narre': 'narre/narre'+mydate+'/narre_130_mean_'+cycle+'z',
        'nww3' : 'wave/nww3/nww3'+mydate+'/nww3'+mydate+'_'+cycle+'z',
        'nam-hires': 'nam/nam'+mydate+'/nam_conusnest_'+cycle+'z',
        'rap'  : 'rap/rap'+mydate+'/rap_'+cycle+'z'
        }

    model_path = model.get(select_model,0)

    print('Retrieving file...')
    url = 'http://nomads.ncep.noaa.gov:9090/dods/'+model_path
    print('URL:', url)

    global file
    file = netCDF4.Dataset(url)

    return file


def gettimes(file):
    """Retrieve valid times and the number of timesteps
    from the model forecast.  Convert to Central Time.
    """
    validtimes = list()
    time = file.variables['time'][:]
    timeunits = file.variables['time'].units
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/Chicago')
    for timesteps,validtime in enumerate(time):
        validtime = num2date(validtime,units=timeunits)
        validtime = validtime.replace(tzinfo=from_zone)
        validtime = validtime.astimezone(to_zone)
        validtime = validtime.strftime('%I %p CT, %A, %B %d, %Y')
        validtimes.append(validtime)

    return timesteps, validtimes


def getnearpos(array, value):
    """Extract nearest array indices from lat/lon points"""
    idx = (np.abs(array - value)).argmin()
    return idx


def mapindex(file, lower_lat, upper_lat, left_lon, right_lon):
    """Define the array indices from the geographical
    domain's latitude and longitude
    """
    lats = file.variables['lat'][:]
    lons = file.variables['lon'][:]
    lower_lat_idx = getnearpos(lats,lower_lat)
    upper_lat_idx = getnearpos(lats,upper_lat)
    left_lon_idx = getnearpos(lons,left_lon)
    right_lon_idx = getnearpos(lons,right_lon)
    lats = lats[lower_lat_idx:upper_lat_idx]
    lons = lons[left_lon_idx:right_lon_idx]
    
    return lats, lons, lower_lat_idx, upper_lat_idx, left_lon_idx, right_lon_idx


def mapdims(llat, ulat, llon, rlon):
    """Calculate centerpoint latitude and longitude,
    as well as the mapwidth and mapheight
    """
    centerpt_lat = (llat + ulat) / 2
    centerpt_lon = (llon + rlon) / 2
    mapwidth = abs((llon - rlon) * 81541)
    mapheight = abs((ulat - llat) * 111092)
    return centerpt_lat, centerpt_lon, mapwidth, mapheight

#def drawlatlon():
    #m.drawparallels(np.arange(20,60,10),labels=[1,1,0,0])
    #m.drawmeridians(np.arange(-120,-60,20),labels=[0,0,0,1])
    

def animated_gif(select_model, i):
    """Generate animated GIF based on the model
    and the number of timesteps from that model
    """
    print('Creating animated GIF')
    call(["convert","-delay","80","-loop","0",""+select_model+"*.png","-delay",
         "200","-loop","0",""+select_model+str(i)+".png","-resize","1024x512",
         ""+select_model+".gif"],cwd=select_model)
    print('Finished creating animated GIF')
    

def mp4(select_model):
    """Generate MPEG4 movie from the static PNG files"""
    print('Creating MP4')
    call(["ffmpeg", "-framerate", "1/1", "-i", ""+select_model+"%02d.png",
         "-vcodec", "libx264", "-crf","25", "-pix_fmt", "yuv420p", "-y", "-vf",
         "scale=1024:512", ""+select_model+".mp4"], cwd=select_model)
    print('Finished creating MP4')


class Maps(object):
    """Weather Variables have each of the following attributes:
    Title:  Banner/graphic title
    Name:  Name of the variable in the netCDF file
    Contour Levels:  Heat map/contour levels that will be plotted
    Contour Colors:  The color corresponding to each contour level
    Legend Label:  A units label for the legend bar
    """
    
    @staticmethod
    def reflectivity():
        varTitle = 'Forecast Radar'
        varName = 'refd1000m'
        clevs = np.arange(5,80,5)
        varContours = ('#29EDEC','#1BA3F2','#0A22E7','#29FD2F','#1EC522','#128E15',
                       '#FFFD38','#E7BE2A','#FD8F25','#FC0D1B','#CA3415','#97040C',
                       '#FC28FC','#983BC9','#FFFFFF',)
        legendLabel = 'Reflectivity (dBZ)'
        return varTitle, varName, clevs, varContours, legendLabel
    
    @staticmethod
    def rainaccumulation():
        varTitle = 'Rainfall Total'
        varName = 'apcpsfc'
        clevs = [0.01,0.05,0.10,0.25,0.50,0.75,1.00,2.00,5.00]
        varContours = ('#edf8fb','#b2e2e2','#66c2a4','#238b45','#fef0d9','#fdcc8a',
                       '#fc8d59','#e34a33','#b30000',)
        legendLabel = 'inches'
        return varTitle, varName, clevs, varContours, legendLabel
        
    @staticmethod
    def windgusts():
        varTitle = 'Wind Gusts'
        varName = 'gustsfc'
        clevs = [20,25,30,35,40,45,50,55,60,65,70,75]
        varContours = ('#f1eef6','#bdc9e1','#74a9cf','#0570b0','#ffffd4','#fed98e',
                       '#fe9929','#cc4c02','#f1eef6','#d7b5d8','#df65b0','#ce1256',)
        legendLabel = 'miles per hour'
        return varTitle, varName, clevs, varContours, legendLabel
    
    @staticmethod
    def snowaccumulation():    
        varTitle = 'Snowfall Total'
        varName = 'apcpsfc'
        clevs = [0.1,0.5,1.0,3.0,6.0,9.0,12.00,18.00,24.00]
        varContours = ('#f1eef6','#bdc9e1','#74a9cf','#0570b0','#feebe2','#fbb4b9',
                        '#f768a1','#c51b8a','#7a0177',)
        legendLabel = 'inches'
        return varTitle, varName, clevs, varContours, legendLabel
    
    @staticmethod
    def local():
        lower_lat = 41.6
        upper_lat = 44.4
        left_lon  = -92.2
        right_lon = -87.0
        return lower_lat, upper_lat, left_lon, right_lon
    
    @staticmethod
    def regional():
        lower_lat = 37.0
        upper_lat = 50.0
        left_lon  = -105.0
        right_lon = -82.0
        return lower_lat, upper_lat, left_lon, right_lon
    
    @staticmethod
    def sectors():
        sectors = {'local':[41.6, 44.4, -92.2, -87.7],
                   'regional':[37.0, 50.0, -105.0, -82.0],
        }
        return sectors